In [ ]:
import os
import sys
import re
import pandas as pd
import datetime
import imaplib
import json
import webbrowser

#need to add current directory to path
sys.path.append(os.path.abspath(os.path.curdir))

In [ ]:
import email
from email.parser import HeaderParser

In [ ]:
#import iMap class
from cleanUp_MailBx import imaplib
from cleanUp_MailBx import iMapMailBox
from cleanUp_MailBx import NETFIRMS_URL
from cleanUp_MailBx import NETFIRMS_USERID_FRM
from cleanUp_MailBx import NETFIRMS_PWD_FRM

from cleanUp_MailBx import GMAIL_URL
from cleanUp_MailBx import GMAIL_USERID_FRM
from cleanUp_MailBx import GMAIL_PWD_FRM

from cleanUp_MailBx import deleteAgedMsg
from cleanUp_MailBx import deleteAged_ByDomain
from cleanUp_MailBx import create_CutOffDt


from oauth2 import GeneratePermissionUrl
from oauth2 import  AuthorizeTokens
from oauth2 import TestImapAuthentication
from oauth2 import GenerateOAuth2String
from oauth2 import RefreshToken
from urllib.parse import urlparse, parse_qs

In [ ]:
print(os.path.abspath(os.path.curdir))


In [ ]:
#DELETE AGED MESSAGES
deleteAgedMsg(NETFIRMS_URL,NETFIRMS_USERID_FRM,NETFIRMS_PWD_FRM,90)

In [ ]:
#DELETE AGED MESSAGES by domain
deleteAged_ByDomain(NETFIRMS_URL,NETFIRMS_USERID_FRM,NETFIRMS_PWD_FRM,30,'electjon.com')

In [ ]:
"""
 ALL THE FUNCTIONS in this cell have been moved to imapLib class and should be called from there.
"""
def ExtractID(srcString):
    regEx_id="^(\d+)[^{]+{(\d+)}"
    match = re.search(regEx_id,srcString)
    if match:
        return match.group(1),match.group(2)
    else:
        return False,False

#reg ex for extracting email===========================
def ExtractAddress(srcString):
    regEx_address="From:([^<]+)<(.+)>"
    match = re.search(regEx_address,srcString)
    if match:
        return match.group(1),match.group(2)
    else:
        return "",srcString
    
def ExtractDate(srcString):
    regEx_Dt="Date:(.+)"
    match = re.search(regEx_Dt,srcString)   
    if match:
        dt = match.group(1)        
        dt= dt.strip() 
        #print(srcString)
        #print(dt)
        try:
            result=datetime.datetime.strptime(dt[0:16],'%a, %d %b %Y')
            return result,srcString
        except ValueError as v:
             return "Date Conversion Error",srcString
    else:
        return "No Date Found",srcString

# Connect to Mailbox
## Select Inbox as working mailbox

# Using non-GMAIL MAILBOX
### Must use user ID/PWD to login

In [ ]:
#log in for non-gmail
mailBox = iMapMailBox()
mailBox.server = NETFIRMS_URL
mailBox.userID =  NETFIRMS_USERID_FRM #NETFIRMS_USERID_FRM
mailBox.password = NETFIRMS_PWD_FRM #NETFIRMS_PWD_FRM
mailBox.createConn()
mailBox.conn.login(NETFIRMS_USERID_FRM, NETFIRMS_PWD_FRM)
rspns,emailCnt = mailBox.conn.select('INBOX')
print(f'email count = {emailCnt[0].decode("utf-8")}')

# Using GMAIL MAILBOX
### Must follow OAUTH2 procedures to generate auth_string

In [ ]:
#URL TO credentials
url_GOOGLE_Cred='env\client_secret_web-app.json'
url_Auth_Tokens= r'env\auth_tokens_frmunin945.json'
url_new_Tokens= r'env\new_tokens_frmunin945.json'

#load google credentials from JSON file
googleCredentials = json.load(open(url_GOOGLE_Cred))
print("="*50)
print("GOOGLE CREDENTIALS")
for key,value in googleCredentials["web"].items():
    print(f" {key} Value:{value}")


auth_tokens = json.load(open(url_Auth_Tokens))
print("="*50)
print("AUTH-TOKENS")
for key,value in auth_tokens.items():
    print(f" {key} :{value}")

## FINAL STEP - for login need AUTH_STRING

In [ ]:
#FINAL STEP of process, generate auth-string
auth_string = GenerateOAuth2String('frmunin945@gmail.com', auth_tokens['access_token'],False)
print(auth_string)

In [ ]:
#log in for gmail -have to use 0auth23
#auth_string must use string generated at the bottom of GMAIL section
mailBox = iMapMailBox()
mailBox.server = GMAIL_URL #GMAIL-URL
mailBox.userID =  GMAIL_USERID_FRM #GMAIL_USERID_FRM
mailBox.password = GMAIL_PWD_FRM
mailBox.createConn()
mailBox.conn.authenticate( mechanism='XOAUTH2', authobject=lambda x: auth_string.encode('utf-8'))
rspns,emailCnt = mailBox.conn.select('INBOX')
print(f'email count = {emailCnt[0].decode("utf-8")}')

# GENERIC code to select INBOX
### Can use the code below on any type of IMAP connection

In [ ]:
#select INBOX again
rspns,emailCnt = mailBox.conn.select('INBOX')
print(f'email count = {emailCnt[0].decode("utf-8")}')

In [ ]:
#select SENT folder and delete all emails 
rspns,emailCnt = mailBox.conn.select('INBOX.Sent')
print(f'email count = {emailCnt[0].decode("utf-8")}')
mailBox.delete_eMails('ALL') #DELETE ALL MAILS IN SENT FOLDER

In [ ]:
#list mailboxes folders
rspns, mailBox_list = mailBox.list_folders()
print(f"{len(mailBox_list)} mailboxes found")
result = []
for mailbox in mailBox_list:
    local =mailbox.decode("utf-8")
    #mailbox has the format (\HasChildren) "." INBOX
    result.append(local.split('"." ')[-1]) #using -1 as index returns last item in list
print(result)    

In [ ]:
folders = mailBox.get_Folder_List()
print(folders)

In [ ]:
typ,dtl = mailBox.conn.search(None,'ALL')
print(dtl)

In [ ]:
#create a dataframe of emails in INBOX, group by FROM Header
#note * stands for highest available number
typ, dtl = mailBox.conn.fetch('1:*', '(FLAGS BODY[HEADER.FIELDS (FROM DATE)])')
data=[]
for ln in dtl:
    if type(ln) is tuple:
        row=[]
        id,from_address = ln
        id1,id2 = ExtractID (id.decode('utf-8'))
        emailAddress, url = ExtractAddress(from_address.decode('utf-8'))
        rcvDt,hdr = ExtractDate(from_address.decode('utf-8'))
        row=([id1,id2,emailAddress,url,rcvDt])   
        data.append(row)

df = pd.DataFrame(data,columns=['id','id2','emailNm','url','rcvDt'])
df.set_index('id',inplace=True)


In [ ]:
#finding totals by url
grp=df.groupby('url')['emailNm'].count().sort_values(ascending=False)
print(grp.head (15))

In [ ]:
#filtering dataframe by url
fltr=df['url'].str.contains('nerdwallet.com')
df_fltrd =df[fltr]

### Delete all emails from sender url

In [ ]:
sender_url ="nypost.com"
rspns,emailCnt = mailBox.conn.select('INBOX')
print(f'Prior to delete {emailCnt[0].decode("utf-8")} emails found')
rspns = mailBox.delete_Selected_eMails(f'(FROM "{sender_url}")')
rspns,emailCnt = mailBox.conn.select('INBOX')
print(f'After delete {emailCnt[0].decode("utf-8")} emails found')

### Create Dataframe of all emails over a certain Age

In [ ]:
age = 90
cutoffDt = create_CutOffDt(age) #cutoff date is any emails older than 90 day
msgIc_List = mailBox.find_eMails_priorToDt(cutoffDt)
print(msgIc_List)
#df = mailBox.create_DF_Sender_ReceiveDt(result)
#df.head(25)

In [ ]:

hdrList = mailBox.create_JSON_Msg_Headers(msgIc_List)
df = pd.DataFrame(hdrList)
df.head()

In [ ]:
grp= df.groupby('domain').size().sort_values(ascending=False)
filtered = grp[grp > 5]
print(filtered)

In [ ]:
grp = mailBox.create_url_List(3,10)
print(f"{len(grp)} url's had more than 10 emails older than 3 days ")
print(grp)
print(f"Total emails older that 3 days old - {grp.sum()}")
print(grp.sum())

In [ ]:
#testing cycling thru series
for url,value in grp.items():
    print(f"{url} {grp.index.get_loc(url)} has {value} emails")

In [ ]:
print(len(hdrList[0]))
cols = list(hdrList[0].keys())
print(cols)
from_fld = hdrList[0]['From']
print (from_fld)
regex_url = r'<(.+)>'
match = re.search(regex_url,from_fld)
if match:
    hdrField_url = match.group(1)
    print(hdrField_url)

In [ ]:
for row_id, row_data in enumerate(hdrList):
    for col_id, field_Name in enumerate(row_data):
        print (row_data[field_Name])

In [ ]:
#finding totals by url
grp = df.groupby('url')['Subject'].count().sort_values(ascending=False)
grp.head(25)

In [ ]:
type(grp)
#apply filter for any url > 1
""" 
The where() method replaces values where the condition is False with NaN (or a specified other value).
You can then drop the NaN values if you only want the filtered elements.
"""
filtered = grp.where(grp > 5).dropna()
for index, value in filtered.items():
    print(f"Index: {index}, Value: {value}")

In [ ]:
df = mailBox.create_DF_Msg_Headers(msgIc_List)
min_count = 5
result = df.groupby('url').size().sort_values(ascending=False)
print(result)      

In [ ]:
#delete aged emails from url
target_url="bjswholesaleclub@em.bjs.com"
url_PART = target_url.split("@",1)
print(url_PART[1])

In [ ]:


#filter for items
fltr = df['url'].str.contains(target_url,regex=True,na=False)
df_filtered = df[fltr]
series_id = df_filtered.index.tolist() 
print(f"{len(series_id)} emails found from {target_url} that were at least {age} days old.")
qry =",".join(series_id)

print(qry)
mailBox.delete_eMails_ById(series_id)

In [ ]:
result = mailBox.find_eMails_byDomain(target_url)
print(result)

In [ ]:
msg=['1','2','3']
df = mailBox.create_DF_Sender_ReceiveDt(msg)
df.head()

In [ ]:
print(len(result_JSON))
print(type(result_JSON))

In [ ]:
#TEST - create json 
msg=['1','2','3']
result_JSON =mailBox.create_JSON_Header_Field(result)
#print(result)
#df = pd.DataFrame(result)
#df.head()
ages = [d['ID'] for d in result_JSON]
print(ages)

In [ ]:
dt=datetime.datetime.strptime(result_lIST[0]['Rcvd_Date'][0:16],'%a, %d %b %Y')
print(dt)
print(type(dt))

In [ ]:
msgid = msgHeader['Message-ID'] 
#'d4dbdeae4dbdba31d21993eca1904075@mywater.veolia.us'
print(msgid)
qry= f"(header Message-ID '{msgid}')"
print(qry)
type,data= mailBox.conn.search(None,qry)
print(type)
print(data)

In [ ]:
typ,data2 = mailBox.conn.fetch(msg,'BODY')
print(data2)

# Working with OAUTH2 
## Use OATH2 to generate auth_string

In [ ]:
## Generate a New Authorization code

Use GeneratePermissionUrl to generate a URL. Use this URL in a browser to grant permission to your app.
Permission comes in the form of Authorization Code. The process above will generate a redirect uri. This uri has the authorization code within it.

https://oauth2.dance/?***code=4/0Ab32j92PCQ6H5kpfm1HEbtisouxFPe3cM7-SajDpTH74y4RQX6DF2I8m_2c1w__0ukv-rA***&scope=https://mail.google.com/

You can manually strip out the code and copy to auth_code variable

In [ ]:
#only needed to generate new credentials - copy to browser to follow
#note authorized token shows up in this URL https://oauth2.dance/
request_url = GeneratePermissionUrl(googleCredentials['web']['client_id'])
#be certain to select the GMAIL account you're looking to edit
webbrowser.open(request_url)
parsed_url = urlparse(request_url)
print(parsed_url.query)

In [ ]:
print(request_url) 
print(parsed_url)

In [ ]:
#the step above will open a page, the side may not be available, but you can extract auth code from the url 
authorization_code="4/0AcvDMrDA3IC8G0kfe0l2nkwou8vdRWXmg2NaJ4HvLrPKI56KN-LQLECP8mSMzbILesrmGA"
auth_code_Dec_2025="4/0ATX87lOFZxH2YbCIJbyYNNCoiPRPVvsbB6_dAUJaNhvbAA65s64S4s856g-DWBEyyWC1vw"

### Generate auth tokens

In [ ]:
auth_tokens = AuthorizeTokens(googleCredentials['web']['client_id'], 
                              googleCredentials['web']['client_secret'],
                              auth_code_Dec_2025)
#saving tokens to S file
if auth_tokens:
    auth_json = json.dumps(auth_tokens, indent=4) 
    # Writing to sample.json
    with open(url_Auth_Tokens, "w") as outfile:
        outfile.write(auth_json)
    print(f'auth tokens exported to {url_Auth_Tokens}')    
    print('AUTH TOKENS')
    for key,value in auth_tokens.items():
        print(f" {key} :{value}")
else:
    print('ERROR  -  No Auth tokens were returned')


## If token needs to be refreshed

In [ ]:
print(f"Refresh Token: {auth_tokens['refresh_token']}")
print(f"Access Token: {auth_tokens['access_token']}")

In [ ]:
#REFRESH ACCESS TOKEN and save oauth token file
auth_tokens = RefreshToken(googleCredentials['web']['client_id'], 
                              googleCredentials['web']['client_secret'],
                               auth_tokens['refresh_token'])
#saving tokens to S file
if auth_tokens:
    auth_json = json.dumps(auth_tokens, indent=4) 
    # Writing to sample.json
    with open(url_new_Tokens, "w") as outfile:
        outfile.write(auth_json)
    print(f'auth tokens exported to {url_new_Tokens}')    
    print('REFRESHED TOKENS')
    print(auth_tokens)
else:
    print('ERROR  -  No Auth tokens were returned')


Working with YAML

In [ ]:
import yaml 

In [ ]:
#get user list from yaml file
with open('env\imap.yaml','r') as file :
      data = yaml.safe_load(file)
for user in data['users']:
    msg = f"{user['name']} has {len(user['mailbox'])} mailboxes"
    print(msg.split(" ", maxsplit=2))
    print(msg.split(" ", maxsplit=2)[0])
    
user = data['users'][0]
print(f"mailbox for user {user['name']}")
for  mailbox in data['users'][0]['mailbox']:
    print(mailbox)